In [351]:
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

from sklearn.feature_extraction.text import TfidfVectorizer

In [107]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
dler = nltk.downloader.Downloader()

dler._update_index()
dler.download('all')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_dat

[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package mwa_ppdb to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mwa_ppdb is already up-to-date!
[nltk_data]    | Downloading package names to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Downloading package nombank.1.0 to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nombank.1.0 is already up-to-date!
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nonbreaking_prefixes is already up-to-date!
[nltk_data]    | Downloading package nps_chat

[nltk_data]    |   Package verbnet is already up-to-date!
[nltk_data]    | Downloading package verbnet3 to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package verbnet3 is already up-to-date!
[nltk_data]    | Downloading package webtext to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package webtext is already up-to-date!
[nltk_data]    | Downloading package wmt15_eval to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wmt15_eval is already up-to-date!
[nltk_data]    | Downloading package word2vec_sample to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package word2vec_sample is already up-to-date!
[nltk_data]    | Downloading package wordnet to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wordnet is already up-to-date!
[nltk_data]    | Downloading package wordnet2021 to
[nltk_data]

True

In [282]:
data = pd.read_excel('./data/data.xlsx')
data

,Case Number,Summary
0,1249923,An appeal by special leave against the judgme...
1,68170353,Drug and Magic Remedies (Objectionable Advert...
2,779491,Criminal Appeal No. 204 of the Calcutta High ...
3,591481,"Petition Nos. 81, 62, 63 & 3 of 1959 was file..."
4,1034384,"Criminal Appeals Nos. 36-D, 37-D and 52-D of ..."
...,...,...
9196,1057848,The appellant was working as Officiating Secr...
9197,550254,The appellants and their deceased father S. B...
9198,1073551,The disputed land belonged originally to one ...
9199,998337,Jaimal was the owner of the suit properties ....



### Data cleaning :

In [283]:
remove_str = " Take notes as you read a judgment using our Virtual Legal Assistant"
drop_indices = data[data['Summary'].apply(lambda x : remove_str in x)].index
print(len(drop_indices))


382


In [284]:
data.drop(index=drop_indices,inplace=True)
data.reset_index(inplace = True)
data

,index,Case Number,Summary
0,0,1249923,An appeal by special leave against the judgme...
1,1,68170353,Drug and Magic Remedies (Objectionable Advert...
2,2,779491,Criminal Appeal No. 204 of the Calcutta High ...
3,3,591481,"Petition Nos. 81, 62, 63 & 3 of 1959 was file..."
4,4,1034384,"Criminal Appeals Nos. 36-D, 37-D and 52-D of ..."
...,...,...,...
8814,9196,1057848,The appellant was working as Officiating Secr...
8815,9197,550254,The appellants and their deceased father S. B...
8816,9198,1073551,The disputed land belonged originally to one ...
8817,9199,998337,Jaimal was the owner of the suit properties ....


In [286]:
data.drop(columns='index',axis=1,inplace=True)

In [287]:
data

,Case Number,Summary
0,1249923,An appeal by special leave against the judgme...
1,68170353,Drug and Magic Remedies (Objectionable Advert...
2,779491,Criminal Appeal No. 204 of the Calcutta High ...
3,591481,"Petition Nos. 81, 62, 63 & 3 of 1959 was file..."
4,1034384,"Criminal Appeals Nos. 36-D, 37-D and 52-D of ..."
...,...,...
8814,1057848,The appellant was working as Officiating Secr...
8815,550254,The appellants and their deceased father S. B...
8816,1073551,The disputed land belonged originally to one ...
8817,998337,Jaimal was the owner of the suit properties ....


### Go to Jagad ----------------

In [430]:
summaries = data['Summary'].tolist()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [314]:
extra_stop_words = ['appeal','petition','civil','appellant','judgment']

### Adding 'court' and 'act' in stopword does not consider tenant and rent in topwords
### Sometimes, for the given examples, the algorithm is assigning wrong clusters during inference. This is because the other cluster might have more weightage for the top keyword like 'court'. This is happening because the other cluster has more data points whereas the target cluster has very less as compared to the other one (Check the number of times top words is present in each of them)
### extra_stop_words = ['appeal','petition','civil','appellant','judgment'] and num_clusters=10
### If we add 'court' to stop words, it is not considering 'tenant' and 'rent' as top words

In [315]:
dler = nltk.downloader.Downloader()

dler._update_index()
dler.download('all')
# Define the preprocessing steps
stop_words = set(stopwords.words('english')+extra_stop_words)

lemmatizer = WordNetLemmatizer()

def preprocess_summary(summary):
    # Tokenize the summary
    words = word_tokenize(summary.lower())

    # Remove stop words and punctuation
    words = [w for w in words if w.isalpha() and w not in stop_words]

    # Lemmatize the words
    words = [lemmatizer.lemmatize(w) for w in words]

    # Join the words back into a string
    preprocessed_summary = ' '.join(words)
    return preprocessed_summary

# Apply the preprocessing steps to all the summaries
preprocessed_summaries = [preprocess_summary(summary) for summary in summaries]


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    | 

[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package mwa_ppdb to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mwa_ppdb is already up-to-date!
[nltk_data]    | Downloading package names to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Downloading package nombank.1.0 to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nombank.1.0 is already up-to-date!
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nonbreaking_prefixes is already up-to-date!
[nltk_data]    | Downloading package nps_chat

[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downloading package universal_tagset to
[nltk_data]    |     C:\Users\nsimh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package universal_tagset is already up-to-date!
[nltk_data]    | Downloading package universal

In [316]:
# preprocessed_summaries


Try with different max_features

Things we can play around with  - max_features for vectorizing, different vectorization methods and different grouping methods

Find the right k for K-means clustering

In [317]:
NO_OF_CLUSTERS = 10
all_stop_words= stopwords.words('english')+extra_stop_words

In [318]:
#Hyperparamater max_features caps the dimentsion
#for 3680 datasamples - the dim of each sample is 9961
#as no of samples increases for training, the dim of each also increases

### ************************************ START OF JUGAD

In [288]:
extra_stop_words = ['appeal','petition','civil','appellant','judgment']
NO_OF_CLUSTERS = 10

In [400]:
class lawRecomm:

    def __init__(self,data,no_of_clusters=10):
        self.NO_OF_CLUSTERS=no_of_clusters
        self.kmeans=KMeans(self.NO_OF_CLUSTERS,random_state=42)
        self.complete_data = data
        self.target_cluster = -1
        self.vectorized_summaries=[]
        
        summaries = data['Summary'].tolist()
        preprocessed_summaries=[self.preprocess_summary(summary) for summary in summaries]
        self.complete_data['PreProcessedSummary']=preprocessed_summaries
        
    def preprocess_summary(self,summary):

        stop_words = set(stopwords.words('english')+extra_stop_words)

        lemmatizer = WordNetLemmatizer()
        # Tokenize the summary
        words = word_tokenize(summary.lower())

        # Remove stop words and punctuation
        words = [w for w in words if w.isalpha() and w not in stop_words]

        # Lemmatize the words
        words = [lemmatizer.lemmatize(w) for w in words]

        # Join the words back into a string
        preprocessed_summary = ' '.join(words)
        return preprocessed_summary

    def predict_cluster(self,sentance):
        
        preprocessed_summaries = self.complete_data['PreProcessedSummary'].tolist()
        preprocessed_sentance = self.preprocess_summary(sentance) #Getting the preprocessed summary
        preprocessed_summaries.append(preprocessed_sentance) #Appending to all summaries

        vectorizer = TfidfVectorizer()# Define the TF-IDF vectorizer
        self.vectorized_summaries = vectorizer.fit_transform(preprocessed_summaries)# Vectorize the preprocessed summaries
#         print(" INFO : Vectorised Summaries - DIMENSION of Each :",vectorized_summaries[0])
       # kmeans = KMeans(n_clusters=self.NO_OF_CLUSTERS, random_state=42)

        cluster_labels = self.kmeans.fit_predict(self.vectorized_summaries)# Apply the K-Means algorithm to the vectorized summaries
        # Evaluate the clustering results using the silhouette score
#         silhouette_avg = silhouette_score(vectorized_summaries, cluster_labels)
#         print(f'Silhouette score: {silhouette_avg:.2f} ')
        
        self.target_cluster = cluster_labels[-1]
        print(" TARGET CLUSTER : ",self.target_cluster)
        
        #Clean up (Due to gudad)
        cluster_labels=cluster_labels[:-1]
        preprocessed_summaries = preprocessed_summaries[:-1]
        
        #Updating data
        self.complete_data['cluster']=cluster_labels
        
        
        return self.vectorized_summaries
        
    def getTopKWords(self): #Gets the top K words in each cluster
        #Initilisation:
        cluster_labels = self.complete_data['cluster']
        preprocessed_summaries = self.complete_data['PreProcessedSummary'].tolist()
        all_stop_words= stopwords.words('english')+extra_stop_words
        # Get the indices of the documents belonging to each cluster
        cluster_indices = [np.where(cluster_labels == i)[0] for i in range(self.NO_OF_CLUSTERS)]
        # Loop over each cluster
        for i, indices in enumerate(cluster_indices):
            # Get the documents belonging to the current cluster
            cluster_documents = [preprocessed_summaries[index] for index in indices]
            #cluster_documents = [summaries[index] for index in indices]


            # Create a TF-IDF vectorizer and fit it on the documents belonging to the current cluster
            vectorizer = TfidfVectorizer(stop_words=all_stop_words)
            tfidf_matrix = vectorizer.fit_transform(cluster_documents)

            # Get the sum of the TF-IDF scores for each word across all the documents belonging to the current cluster
            word_scores = np.asarray(tfidf_matrix.sum(axis=0)).ravel()
            sorted_word_indices = np.argsort(word_scores)[::-1]

            # Print the top 10 words for the current cluster
            print(f"\nTop 10 words for Cluster {i}:")
            feature_names = np.array(vectorizer.get_feature_names_out())
            for j in range(10):
                print(f"{feature_names[sorted_word_indices[j]]} ({word_scores[sorted_word_indices[j]]:.2f})")

    def getCompleteData(self):
        return self.complete_data
    
    def getTargetCluster(self):
        return self.target_cluster
    
    def getSummartOfCluster(self,cluster_no,Context="Summary"):
        '''Prints the Requited Content for the given cluster
            Context :
                1) "Summary" - For Summary of the document (By default)
                2)  "PreProcessedSummary" - To see the prepreocessed summary of the data
            
        '''
        for sum in self.complete_data[self.complete_data['cluster']==cluster_no][Context]:
            print("-----------")
            print(sum)
            
    def getTopXDoc(self,no_of_doc,distance_measure="euc"):
        '''To get the Top X Documents similar to the test document - 
        Distance Metric-
            "cos" :Cosine Simalarity
            "euc" :Eucledian Distance
        By default considers Cosine Simalarity
            '''
        # Get the cluster label for the new document
        
        vectorized_summaries = self.vectorized_summaries[:-1]
        new_document_vectorized=self.vectorized_summaries[-1]
        new_doc_cluster_label = self.target_cluster
        cluster_labels = self.complete_data['cluster']
        # Get the cluster label for the new document

        # Get the indices of all data points that belong to the same cluster as the new document
        cluster_indices = np.where(cluster_labels == new_doc_cluster_label)[0]
        # Select the corresponding data points
        cluster_data_points = vectorized_summaries[cluster_indices]
        # Compute the cosine similarity between the new document and all other documents in the cluster
        if distance_measure=="euc":
            similarities = euclidean_distances(new_document_vectorized, cluster_data_points)
        else:
            similarities = cosine_similarity(new_document_vectorized, cluster_data_points)

        # Get the indices of the documents sorted by similarity
        sorted_indices = np.argsort(similarities)[0][::-1]

        # Get the top 5 indices
        top_indices = sorted_indices[:no_of_doc+1]

        # Get the indices of the documents in the original dataset
        document_indices = cluster_indices[top_indices]

        # Print the document numbers that are closest
        print(f'Top {no_of_doc} nearest documents in cluster {new_doc_cluster_label }:')
        for document_index in document_indices:
            print("https://indiankanoon.org/doc/"+str(data.iloc[document_index]['Case Number']))
        



In [401]:
obj = lawRecomm(data,NO_OF_CLUSTERS)

In [402]:
obj.getTopKWords()


Top 10 words for Cluster 0:
court (47.61)
singh (47.40)
section (44.35)
high (40.77)
convicted (33.21)
sentenced (31.89)
sentence (29.20)
imprisonment (28.72)
accused (28.39)
code (28.24)

Top 10 words for Cluster 1:
election (23.57)
court (15.75)
high (14.74)
respondent (12.93)
candidate (10.10)
constituency (9.62)
vote (9.19)
declared (9.15)
assembly (8.72)
act (8.45)

Top 10 words for Cluster 2:
court (115.04)
high (98.47)
act (78.50)
respondent (71.51)
company (61.15)
state (59.68)
order (57.35)
case (56.19)
india (43.45)
two (42.24)

Top 10 words for Cluster 3:
court (35.37)
high (30.51)
order (23.18)
dated (22.97)
delivered (18.88)
special (17.36)
leave (15.86)
act (13.43)
madras (12.42)
two (12.20)

Top 10 words for Cluster 4:
land (36.59)
act (24.24)
court (23.59)
high (21.65)
tenant (18.28)
respondent (17.21)
acquisition (12.49)
dated (11.88)
rent (11.86)
order (11.66)

Top 10 words for Cluster 5:
petitioner (30.99)
order (26.94)
act (26.27)
detention (25.11)
constitution (24

In [403]:
new_doc = "Mr.Anand commmited crime that led to the disqualification of him where he murdered him in broad day light for which he was convicted and sentenced"
new_doc = "He and his brother are having dispute over the land they own with the workmen and labours"
new_doc = " The impugned Acts infringe the fundamental rights under Art. (1) of the petitioners who are butchers, gut merchants, curers and cattle dealers to carry on their respective trades . They also contravene the religious practice of the Petitioners' community to sacrifice cows, cattle and sheep and goats  "
new_doc = "The industrial men harrased the labour so an act was passed"
new_doc = "He did not pay tax"
new_doc = "Landlord high tenant rent "
new_doc = "He was arrested on charges of corruption"
new_doc = "The landlord sued the tenant for not paying rent on time"


In [404]:
vectors_summ = obj.predict_cluster(new_doc)
target = obj.getTargetCluster()

C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


 TARGET CLUSTER :  4


In [405]:
obj.getTopXDoc(no_of_doc=5,distance_measure="cos")

Top 5 nearest documents in cluster 4:
https://indiankanoon.org/doc/1371585
https://indiankanoon.org/doc/769101
https://indiankanoon.org/doc/1872225
https://indiankanoon.org/doc/1472953
https://indiankanoon.org/doc/1612858
https://indiankanoon.org/doc/1106648


In [406]:
obj.getTopXDoc(no_of_doc=5,distance_measure="euc")

Top 5 nearest documents in cluster 4:
https://indiankanoon.org/doc/175295508
https://indiankanoon.org/doc/287561
https://indiankanoon.org/doc/1319210
https://indiankanoon.org/doc/1470235
https://indiankanoon.org/doc/1713470
https://indiankanoon.org/doc/1309889


#### Observation : Eucledian Distance simalarity is not upto the mark haha. Cosine Sim is really good

In [ ]:
obj.getSummartOfCluster(4)

In [373]:
vectors_summ

<8820x17033 sparse matrix of type '<class 'numpy.float64'>'
	with 213199 stored elements in Compressed Sparse Row format>

In [377]:
data = obj.getCompleteData()

In [378]:
data[data['cluster']==4]

,Case Number,Summary,PreProcessedSummary,cluster
17,457935,Civil Appeal No.327 of 1955: Civil Appeal no....,relates bombay taluqdari tenure tenure aboliti...,4
145,1604881,Civil Appeal No. 347 of 1955 . Appeal was bro...,brought hindusthan insurance society calcutta ...,4
155,1490310,The suit out of which this appeal arises conc...,suit arises concern right certain plot land la...,4
218,147362898,The former State of Rampur was an independent...,former state rampur independent state theparam...,4
221,111667882,Petition challenges the constitutionality of ...,challenge constitutionality uttar pradesh cons...,4
...,...,...,...,...
8773,440827,Appeal No. 1010 of 1967 relates to a plot of ...,relates plot land numbering vijayapuram villag...,4
8775,672819,Wazir Singh challenged the gift of the land w...,wazir singh challenged gift land asserted belo...,4
8797,418852,Two plots of land bearing No. 936 of Mouza As...,two plot land bearing mouza asansol acquired l...,4
8813,1615625,An appeal by special leave was brought by the...,special leave brought legal representative dew...,4


In [379]:
ind = data[data['cluster']==4].index
ind

Int64Index([  17,  145,  155,  218,  221,  249,  261,  281,  348,  393,
            ...
            8698, 8720, 8721, 8727, 8731, 8773, 8775, 8797, 8813, 8816],
           dtype='int64', length=519)

In [381]:
def vc(vec1,vec2):
    sum=0
    for i in (vec1-vec2).data:
        sum+=i*i
    return sum

d = dict()
for i in ind:
    distance = (vc(vd[-1],vd[i]))
    if(distance in d.keys()):
        d[distance].append(data.iloc[i]['Case Number'])
    else:
        d[distance]=[i]
    
ll =(sorted(d.items()))

In [382]:
count=0
for (key,val) in ll:
    if count>5:
        break
    for ele in val:
        count+=1
        print("https://indiankanoon.org/doc/"+str(data.iloc[ele]['Case Number']))

https://indiankanoon.org/doc/1371585
https://indiankanoon.org/doc/769101
https://indiankanoon.org/doc/1872225
https://indiankanoon.org/doc/1472953
https://indiankanoon.org/doc/1612858
https://indiankanoon.org/doc/1106648


In [236]:
vd

<8820x17033 sparse matrix of type '<class 'numpy.float64'>'
	with 213199 stored elements in Compressed Sparse Row format>

In [237]:
our_v = vd[-1]
our_v - vd[6]

<1x17033 sparse matrix of type '<class 'numpy.float64'>'
	with 48 stored elements in Compressed Sparse Row format>

In [410]:
# list of summaries of documents to be clustered
summaries =obj.getCompleteData()['Summary'].tolist()
summaries

[' An appeal by special leave against the judgment and order of the Punjab High Court dated May 17, 1956 . The appellant was a Naib-Tehsildar at Ferozepur Jhirka in the district of Gurgaon . The allegation against the appellant was that during his tours in the several villages, his son who was a Director of the Starline Pictures Ltd., a film company of Delhi, accompanied him and at the time of mutations the appellant asked the parties whose mutation he was attesting to purchase shares in his son .',
 ' Drug and Magic Remedies (Objectionable Advertisement) Act (XXI of 1954) was passed on April 30, 1954, came into force on April 1, 1955 . Petitioners in Writ Petition No. 81 of 1959, the Hamdard Dawakhana (Wakf) and another, alleged they experienced difficulty in the matter of publicity for their products .',
 ' Criminal Appeal No. 204 of the Calcutta High Court . Appeal by special leave from the judgment and order dated June 23, 1959 . The case against Kangsari Haldar and Jogendra Nath G

In [436]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors



# create a TfidfVectorizer object to convert summaries into feature vectors
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(summaries)

# create a DBSCAN object and fit it to the data
dbscan = DBSCAN(eps=0.5, min_samples=2)
dbscan.fit(X)

# find the nearest summaries to a given summary
given_summary = new_doc

k = 5

# vectorize the given summary
given_X = vectorizer.transform([given_summary])

# find the indices of the k nearest neighbors to the given summary
nn = NearestNeighbors(n_neighbors=k, metric='euclidean')
nn.fit(X)
distances, indices = nn.kneighbors(given_X)

# print the nearest summaries
print(f"The {k} nearest summaries to '{given_summary}' are:")
for i in indices[0]:
    print(f"\t{summaries[i]}")


The 5 nearest summaries to 'The landlord sued the tenant for not paying rent on time' are:
	 The case involved a landlord and a tenant in a shop in Santa Cruz, Bombay . The landlord and the tenant were unable to pay the rent of the landlord . The tenant did not pay the landlord's rent and the landlord was forced to pay it back . The case was brought by the Bombay Rents, Hotel and Lodging House Rates Control (3) of 1947)
	 The landlord sued for eviction on the ground of arrears of rent as provided in s. 7-C(1) of the United Provinces (Temporary) Control of  Rent and. Â  Â  Â  Â  Â  Â  Â  Â 	Eviction  Act, 1947 . The parties were living in adjacent premises and the landlord was in a criminal case by the tenant against the landlord in 1969 for offences under .
	 Civil Appeal No. 499 of 1966: Landlord's appeal against order dismissing tenant's eviction petition . Landlord claimed rent for premises from October 1, 1959 to June 30, 1961 had not been paid . The tenant claimed that this was a 

In [429]:
given_X

<1x21795 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

 ### **************************************** END OF JUGAD

In [417]:
preprocessed_summaries  = obj.getCompleteData()['PreProcessedSummary'].tolist()
preprocessed_summaries

['special leave order punjab high court dated may ferozepur jhirka district gurgaon allegation tour several village son director starline picture film company delhi accompanied time mutation asked party whose mutation attesting purchase share son',
 'drug magic remedy objectionable advertisement act xxi passed april came force april petitioner writ hamdard dawakhana wakf another alleged experienced difficulty matter publicity product',
 'criminal calcutta high court special leave order dated june case kangsari haldar jogendra nath guria hereinafter called appellant heard mitter bhattacharya jj difference opinion learned judge',
 'no filed art constitution india petitioner alleged various action taken authority violated fundamental right act passed april came force april along rule made thereunder',
 'criminal appeal no quashed punjab high court punjab high court bhana mal gulzari mal steel company company three director general manager two salesman respondent accused selling old stock 

In [437]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Vectorize the preprocessed summaries

vectorized_summaries = vectorizer.fit_transform(preprocessed_summaries)
kmeans = KMeans(n_clusters=NO_OF_CLUSTERS, random_state=42)
new_doc_vect = vectorizer.transform([new_doc])

# Apply the K-Means algorithm to the vectorized summaries
cluster_labels = kmeans.fit_predict(vectorized_summaries)

# Evaluate the clustering results using the silhouette score
silhouette_avg = silhouette_score(vectorized_summaries, cluster_labels)
print(f'Silhouette score: {silhouette_avg:.2f} ')



C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Silhouette score: 0.01 


In [438]:
vectorized_summaries

<8819x16864 sparse matrix of type '<class 'numpy.float64'>'
	with 200090 stored elements in Compressed Sparse Row format>

In [439]:
new_doc_vect

<1x16864 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [426]:
vectorized_summaries[0]

<1x16864 sparse matrix of type '<class 'numpy.float64'>'
	with 29 stored elements in Compressed Sparse Row format>

In [327]:
new_document='Mr.Anand commmited crime that led to the disqualification of him where he murdered him in broad day light'

In [328]:
new_doc_preprocess = preprocess_summary(new_document)
new_doc_preprocess

'commmited crime led disqualification murdered broad day light'

In [329]:
# vectorizer = TfidfVectorizer(stop_words="english")
new_document_vectorized = vectorizer.fit_transform([new_doc_preprocess])
new_document_vectorized

<1x8 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [330]:
# # vectorizer = TfidfVectorizer(stop_words="english")



if new_document_vectorized.shape[1] < kmeans.cluster_centers_.shape[1]:
    new_document_vectorized = np.hstack((new_document_vectorized.toarray(), np.zeros((new_document_vectorized.shape[0], kmeans.cluster_centers_.shape[1]-new_document_vectorized.shape[1]))))

kmeans.predict(new_document_vectorized)

array([0])

In [346]:
data[data['cluster']==1]

,Case Number,Summary,cluster,PreProSum
2,779491,Criminal Appeal No. 204 of the Calcutta High ...,1,criminal calcutta high court special leave ord...
9,1092568,Civil Appeal No. 230 of 1959 of 1959 was Civi...,1,order dated october punjab high court writ cou...
27,38328,The High Court held that the rule was ultra v...,1,high court held rule ultra vires offending art...
28,1222631,The Bombay High Court granted a certificate f...,1,bombay high court granted certificate high cou...
38,491029,Civil Appeal No. 342 of 1959: Appeal by speci...,1,special leave decree dated calcutta high court...
...,...,...,...,...
9174,1486515,Civil Appeal No. 24 of 1967: Civil Appeal of ...,1,special leave calcutta high court answering fo...
9181,1548537,Civil Appeal No. 646 of 1967: Civil Appeal of...,1,overturned allahabad high court decision set a...
9182,1179086,An appeal was heard by the Bombay High Court ...,1,heard bombay high court first laxmidas raghuna...
9190,1600971,The State of Assam and Nagaland High Court in...,1,state assam nagaland high court rule rameshwar...


In [432]:
# data['cluster']=cluster_labels
# data['PreProSum']=preprocessed_summaries
# for sum in data[data['cluster']==1]['Summary']:
#     print(sum)
#     print('-----------------')

In [352]:


# Get the indices of the documents belonging to each cluster
cluster_indices = [np.where(cluster_labels == i)[0] for i in range(NO_OF_CLUSTERS)]
# Loop over each cluster
for i, indices in enumerate(cluster_indices):
    # Get the documents belonging to the current cluster
    cluster_documents = [preprocessed_summaries[index] for index in indices]
    #cluster_documents = [summaries[index] for index in indices]

    
    # Create a TF-IDF vectorizer and fit it on the documents belonging to the current cluster
    vectorizer = TfidfVectorizer(stop_words=all_stop_words)
    tfidf_matrix = vectorizer.fit_transform(cluster_documents)
    
    # Get the sum of the TF-IDF scores for each word across all the documents belonging to the current cluster
    word_scores = np.asarray(tfidf_matrix.sum(axis=0)).ravel()
    sorted_word_indices = np.argsort(word_scores)[::-1]
    
    # Print the top 10 words for the current cluster
    print(f"\nTop 10 words for Cluster {i}:")
    feature_names = np.array(vectorizer.get_feature_names_out())
    for j in range(10):
        print(f"{feature_names[sorted_word_indices[j]]} ({word_scores[sorted_word_indices[j]]:.2f})")



Top 10 words for Cluster 0:
court (47.61)
singh (47.40)
section (44.35)
high (40.77)
convicted (33.21)
sentenced (31.89)
sentence (29.20)
imprisonment (28.72)
accused (28.39)
code (28.24)

Top 10 words for Cluster 1:
election (23.57)
court (15.75)
high (14.74)
respondent (12.93)
candidate (10.10)
constituency (9.62)
vote (9.19)
declared (9.15)
assembly (8.72)
act (8.45)

Top 10 words for Cluster 2:
court (115.04)
high (98.47)
act (78.50)
respondent (71.51)
company (61.15)
state (59.68)
order (57.35)
case (56.19)
india (43.45)
two (42.24)

Top 10 words for Cluster 3:
court (35.37)
high (30.51)
order (23.18)
dated (22.97)
delivered (18.88)
special (17.36)
leave (15.86)
act (13.43)
madras (12.42)
two (12.20)

Top 10 words for Cluster 4:
land (36.59)
act (24.24)
court (23.59)
high (21.65)
tenant (18.28)
respondent (17.21)
acquisition (12.49)
dated (11.88)
rent (11.86)
order (11.66)

Top 10 words for Cluster 5:
petitioner (30.99)
order (26.94)
act (26.27)
detention (25.11)
constitution (24

### ----------------------------------------------------------------------------

In [386]:
NO_OF_CLUSTERS=5

In [388]:
vectorised_data = [] #Vector Data Matrix
vectorizer = TfidfVectorizer()
maxL=0
for summary in preprocessed_summaries:
    new_document_vectorized = vectorizer.fit_transform([summary])
    new_document_vectorized = np.hstack((new_document_vectorized.toarray(), np.zeros((new_document_vectorized.shape[0], 52-new_document_vectorized.shape[1]))))
    vectorised_data.append(new_document_vectorized[0])


In [389]:
vectorised_data

[array([0.16222142, 0.16222142, 0.16222142, 0.16222142, 0.16222142,
        0.16222142, 0.16222142, 0.16222142, 0.16222142, 0.16222142,
        0.16222142, 0.16222142, 0.16222142, 0.16222142, 0.16222142,
        0.16222142, 0.16222142, 0.32444284, 0.16222142, 0.16222142,
        0.16222142, 0.16222142, 0.16222142, 0.16222142, 0.16222142,
        0.32444284, 0.16222142, 0.16222142, 0.16222142, 0.16222142,
        0.16222142, 0.16222142, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]),
 array([0.19611614, 0.19611614, 0.19611614, 0.19611614, 0.39223227,
        0.19611614, 0.19611614, 0.19611614, 0.19611614, 0.19611614,
        0.19611614, 0.19611614, 0.19611614, 0.19611614, 0.19611614,
        0.19611614, 0.19611614, 0.19611614, 0.19611614, 0.19611614,
        0.1961

In [390]:
kmeans = KMeans(n_clusters=NO_OF_CLUSTERS, random_state=42)

# Apply the K-Means algorithm to the vectorized summaries
cluster_labels = kmeans.fit_predict(vectorised_data)
cluster_labels

C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=15.
  warnings.warn(


array([3, 1, 4, ..., 3, 4, 3])

In [431]:
# data['cluster']=cluster_labels
# data['PreProSum']=preprocessed_summaries
# for sum in data[data['cluster']==1]['PreProSum']:
#     print(sum)
#     print('-----------------')

In [371]:
new_document="high court reversed decree subordinate judge motihari dated march suit filed present appellant declaration title odd bighas ryotikasht land possession thereof defendant"


In [372]:
preprocess_summary_new_doc = preprocess_summary(new_document)
new_document_vectorized = vectorizer.fit_transform([preprocess_summary_new_doc])
new_document_vectorized = np.hstack((new_document_vectorized.toarray(), np.zeros((new_document_vectorized.shape[0], 52-new_document_vectorized.shape[1]))))

In [373]:
kmeans.predict(new_document_vectorized)

array([1])

In [351]:
new_document_vectorized.shape[1]

5000

### GET TOP words in each cluster

In [352]:

# # Fit the K-Means object on the vectorized summaries to cluster them
# cluster_labels = kmeans.fit_predict(vectorized_summaries)

# Get the indices of the documents belonging to each cluster
cluster_indices = [np.where(cluster_labels == i)[0] for i in range(NO_OF_CLUSTERS)]

# Loop over each cluster
for i, indices in enumerate(cluster_indices):
    # Get the documents belonging to the current cluster
    cluster_documents = [preprocessed_summaries[index] for index in indices]
    
    # Create a TF-IDF vectorizer and fit it on the documents belonging to the current cluster
    vectorizer = TfidfVectorizer(stop_words=all_stop_words)
    tfidf_matrix = vectorizer.fit_transform(cluster_documents)
    
    # Get the sum of the TF-IDF scores for each word across all the documents belonging to the current cluster
    word_scores = np.asarray(tfidf_matrix.sum(axis=0)).ravel()
    sorted_word_indices = np.argsort(word_scores)[::-1]
    
    # Print the top 10 words for the current cluster
    print(f"\nTop 10 words for Cluster {i+1}:")
    feature_names = np.array(vectorizer.get_feature_names_out())
    for j in range(10):
        print(f"{feature_names[sorted_word_indices[j]]} ({word_scores[sorted_word_indices[j]]:.2f})")



Top 10 words for Cluster 1:
court (39.43)
high (33.97)
delivered (21.08)
dated (17.38)
order (16.92)
leave (15.60)
special (14.89)
act (14.21)
bombay (13.97)
case (12.66)

Top 10 words for Cluster 2:
court (47.49)
service (42.31)
alert (41.21)
query (41.14)
high (40.68)
act (27.71)
order (25.65)
one (24.84)
month (22.58)
assistant (22.37)

Top 10 words for Cluster 3:
court (42.75)
high (36.96)
order (24.07)
act (22.46)
dated (19.94)
special (18.65)
leave (18.49)
india (17.06)
respondent (17.05)
delivered (16.73)

Top 10 words for Cluster 4:
court (24.97)
high (21.35)
respondent (17.71)
act (16.92)
order (16.74)
singh (13.41)
dated (13.33)
two (13.07)
state (12.73)
bombay (11.78)

Top 10 words for Cluster 5:
court (40.00)
high (33.83)
order (24.40)
act (24.27)
respondent (21.47)
dated (18.80)
state (17.95)
bombay (16.97)
special (16.77)
year (16.64)


In [212]:
new_doc =" convicted code sentence court high section"

In [256]:
# # Vectorize the new document using the same vectorizer used for the original documents
# new_doc_vectorized = vectorizer.transform([new_doc])

# # Use the trained K-Means model to predict the cluster label for the new document
# predicted_cluster_label = kmeans.predict(new_doc_vectorized)

# # Assign the new document to the predicted cluster
# new_doc_cluster = cluster_documents[predicted_cluster_label[0]]
# new_doc_cluster.append(new_doc)

# Load the trained KMeans model and vectorizer
# with open('kmeans_model.pkl', 'rb') as f:
#     kmeans_model = pickle.load(f)
# with open('vectorizer.pkl', 'rb') as f:
#     vectorizer = pickle.load(f)

# Load the new document
new_document = new_doc

# Vectorize the new document
new_document_vectorized = vectorizer.transform([new_document])

# Ensure that the new document has the same number of features as the training data
if new_document_vectorized.shape[1] < kmeans.cluster_centers_.shape[1]:
    new_document_vectorized = np.hstack((new_document_vectorized.toarray(), np.zeros((new_document_vectorized.shape[0], kmeans.cluster_centers_.shape[1]-new_document_vectorized.shape[1]))))

# Predict the cluster label of the new document
cluster_label = kmeans.predict(new_document_vectorized)

print('The new document belongs to cluster:', cluster_label[0])


The new document belongs to cluster: 0


In [308]:
vectorized_summaries

<3680x9961 sparse matrix of type '<class 'numpy.float64'>'
	with 87257 stored elements in Compressed Sparse Row format>

In [309]:
for i in range(2,100):
# Define the K-Means clustering algorithm
    kmeans = KMeans(n_clusters=i, random_state=42)

    # Apply the K-Means algorithm to the vectorized summaries
    cluster_labels = kmeans.fit_predict(vectorised_data)

    # Evaluate the clustering results using the silhouette score
    silhouette_avg = silhouette_score(vectorised_data, cluster_labels)
    print(f'Silhouette score: {silhouette_avg:.2f} | no of clusters : {i}')


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=15.
  warnings.warn(


Silhouette score: 0.21 | no of clusters : 2


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=15.
  warnings.warn(


Silhouette score: 0.19 | no of clusters : 3


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=15.
  warnings.warn(


...

Silhouette score: 0.11 | no of clusters : 26


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=15.
  warnings.warn(


Silhouette score: 0.11 | no of clusters : 27


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=15.
  warnings.warn(


KeyboardInterrupt: 

In [200]:
for i in range(2,20):
# Define the K-Means clustering algorithm
    kmeans = KMeans(n_clusters=i, random_state=42)

    # Apply the K-Means algorithm to the vectorized summaries
    cluster_labels = kmeans.fit_predict(vectorized_summaries)

    # Evaluate the clustering results using the silhouette score
    silhouette_avg = silhouette_score(vectorized_summaries, cluster_labels)
    print(f'Silhouette score: {silhouette_avg:.2f} | no of clusters : {i}')


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Silhouette score: 0.04 | no of clusters : 2


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Silhouette score: 0.03 | no of clusters : 3


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

Silhouette score: 0.04 | no of clusters : 17


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Silhouette score: 0.04 | no of clusters : 18


C:\Users\nsimh\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Silhouette score: 0.04 | no of clusters : 19


In [ ]:
# Print the top key words for each cluster
n_top_keywords = 10
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer

In [ ]:
data['cluster']=cluster_labels
data['PreProSum']=preprocessed_summaries

In [ ]:
data

In [ ]:
data[data['cluster']==4]['Summary'][:10]

In [ ]:
for sum in data[data['cluster']==4]['PreProSum']:
    print(sum)
    print('-----------------')

In [ ]:
 import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string

# Define the vectorizer
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english') + list(string.punctuation))

# Vectorize the documents
X = vectorizer.fit_transform(data)

# Cluster the documents
n_clusters = 5
km = KMeans(n_clusters=n_clusters, random_state=0)
km.fit(X)

# Get the cluster labels for each document
labels = km.labels_

# Get the text for each cluster
cluster_text = ['' for _ in range(n_clusters)]
for i, text in enumerate(data):
    cluster_text[labels[i]] += ' ' + text

# Clean the text and compute tf-idf scores
cleaned_text = [' '.join(word for word in text.lower().split() if word not in stopwords.words('english') and word not in string.punctuation) for text in cluster_text]
X_cluster = vectorizer.transform(cleaned_text)
tfidf_scores = np.asarray(X_cluster.mean(axis=0)).ravel()

# Print the top 10 keywords for each cluster
for i in range(n_clusters):
    start = i * vectorizer.get_feature_names()
    end = (i + 1) * vectorizer.get_feature_names()
    indices = tfidf_scores[start:end].argsort()[::-1][:10]
    keywords = [vectorizer.get_feature_names()[i] for i in indices]
    print(f'Cluster {i}: {keywords}')


In [ ]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Define the number of clusters
num_clusters = 5

# Create a K-Means clustering object
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Fit the K-Means object on the vectorized summaries to cluster them
cluster_labels = kmeans.fit_predict(vectorized_summaries)

# Get the indices of the documents belonging to each cluster
cluster_indices = [np.where(cluster_labels == i)[0] for i in range(num_clusters)]

# Loop over each cluster
for i, indices in enumerate(cluster_indices):
    # Get the documents belonging to the current cluster
    cluster_documents = [summaries[index] for index in indices]
    
    # Create a TF-IDF vectorizer and fit it on the documents belonging to the current cluster
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(cluster_documents)
    
    # Get the sum of the TF-IDF scores for each word across all the documents belonging to the current cluster
    word_scores = np.asarray(tfidf_matrix.sum(axis=0)).ravel()
    sorted_word_indices = np.argsort(word_scores)[::-1]
    
    # Print the top 10 words for the current cluster
    print(f"\nTop 10 words for Cluster {i+1}:")
    feature_names = np.array(vectorizer.get_feature_names())
    for j in range(10):
        print(f"{feature_names[sorted_word_indices[j]]} ({word_scores[sorted_word_indices[j]]:.2f})")


ISSUES : 
1) All getting into same cluster due to dim mismatch

## ----------------------------------- 

## -------------------------------------------------------------- WORD2VEC 

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


In [ ]:
documents = summaries
processed_docs = [word_tokenize(doc.lower()) for doc in documents]


In [ ]:
model = Word2Vec(processed_docs, window=5, min_count=1, workers=4)


In [ ]:
# Get the vector representation of a word
vector = model.wv['word']

# Get the vector representation of a document summary
doc_vector = model.infer_vector(word_tokenize('Document summary'))


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

# Create a list of tagged documents
tagged_docs = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[i]) for i, doc in enumerate(summaries)]

# Train the Doc2Vec model
model = Doc2Vec(tagged_docs, vector_size=100, window=5, min_count=1, workers=4)

# Infer a vector representation for a new document
new_doc_vector = model.infer_vector(word_tokenize('Document summary'))

# Find similar documents to the new document
similar_docs = model.docvecs.most_similar([new_doc_vector])


In [ ]:
similar_docs


In [ ]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
def preprocess_text(text):
    # your preprocessing code here
    return text


In [ ]:
# model = Word2Vec.load('path/to/word2vec/model')

In [ ]:
def get_document_vector(document):
    # preprocess the document
    preprocessed_doc = preprocess_text(document)
    # split the preprocessed document into words
    words = preprocessed_doc.split()
    # remove words that are not in the Word2Vec model's vocabulary
    words = [word for word in words if word in model.wv.vocab]
    # calculate the average of all word vectors in the document
    if len(words) > 0:
        vector = np.mean(model[words], axis=0)
    else:
        vector = np.zeros((model.vector_size,))
    return vector


In [ ]:
def perform_clustering(documents, num_clusters):
    # get the vector representation of each document
    document_vectors = [get_document_vector(doc) for doc in documents]
    # perform K-Means clustering
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(document_vectors)
    # return the cluster labels for each document
    return kmeans.labels_


In [ ]:
documents = summaries
num_clusters = 2
cluster_labels = perform_clustering(documents, num_clusters)


In [ ]:
from gensim.models import Word2Vec,Doc2Vec
from nltk.tokenize import word_tokenize


In [ ]:
documents = summaries
processed_docs = [word_tokenize(doc.lower()) for doc in documents]


In [ ]:
model = Word2Vec(processed_docs, vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
# Get the vector representation of a word
vector = model.wv['word']

# Get the vector representation of a document summary
doc_vector = model.infer_vector(word_tokenize('Document summary'))


In [ ]:
# Find similar words to 'word'
similar_words = model.wv.most_similar('word')

# Find similar document summaries to 'Document summary 1'
similar_docs = model.docvecs.most_similar([model.infer_vector(word_tokenize('Document summary 1.'))])


In [ ]:
!word2vec --version